In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random  # get x random LSOAs
import time

import seaborn as sns

sns.set_style('whitegrid')

from Segmentation.utils.Segmentation_useful_functions import *

from Segmentation.utils.Calibration_functions import *

# data path for bulky data not in online repo
DataPath = "C:/Users/cenv0795/Data/STEVE_DATA/Scen_CarSegments/"

# load NewCars_per_LSOA
f = open("./data/Cars_per_LSOA.pckl", "rb")
Cars_per_LSOA = pickle.load(f)
f.close()

LSOAs = Cars_per_LSOA.GEO_CODE.unique().tolist()
years = range(2012, 2036)

#everything is driven by NewCars_per_LSOA. Select only a subset and everything produced will be a subset
NewCars_per_LSOA = Cars_per_LSOA[['GEO_CODE'] + ['NewCars'+str(y) for y in years]]
TotalCars_per_LSOA = Cars_per_LSOA[['GEO_CODE'] + ['TotalCars'+str(y) for y in years]]

# ---- AGE DATA FOR SCRAPPAGE MODULE ---- #
# load age data
f = open("./data/AgeProfile_by_LSOA_normalised.pckl", 'rb')
AgeData = pickle.load(f)
f.close()

# load delta lookup
f = open("./data/delta_AvgAge_lookup.pckl", 'rb')
AvgAge_d_lookup = pickle.load(f)
f.close()


In [3]:
LED_scenario = "LED2"

# Cost_data
Cost_Data = pd.read_csv(f"./tables/Cost_Data_{LED_scenario}.csv")

# Scen_CarSegments - by LSOA - for Scotland
f = open(f"{DataPath}Scen_CarSegments_byLSOA_SCOTLAND_{LED_scenario}.pckl", "rb")
Scen_CarSegments = pickle.load(f)  # LSOA-specific
f.close()

# Fuel
CT_Fuel = pd.read_csv("./tables/CT_Fuel.csv")  # fuel lookup

# Technology
Technology = pd.read_csv(f"./tables/Technology_{LED_scenario}.csv")

# exclude other vehicle types (cars only)
Technology = Technology[Technology.VehTypeID == 3]

# replace na with 2050 (for final year)
Technology.Final_Year.fillna(int(2050), inplace=True)

# only used variables (see Segmentation_useful_functions.py)
Technology = Technology[['TechID', 'Availability', 'Final_Year', 'MassCatID', 'HybridFlag', 'FuelID']]

# Cost_Data contains all necessary info on technologies. Isolate so it's just cars up to the final year
Cost_Data = Cost_Data[(Cost_Data.VehTypeID == 3) & (Cost_Data.Year <= years[-1])]  # VehTypeID = 3 for cars

# only used variables (see Segmentation_useful_functions.py)
Cost_Data = Cost_Data[['TechID', 'Year', 'Y1Costs', 'AnnCostsPriv', 'ChargingTime', 'SupplyPenalty', 'Range',
                       'NoChargingAccessPenalty']]

In [4]:
# Add columns for indiviuals who are not aware of EV incentives, without charging access
Scen_CarSegments['shNotAwareEV'] = 1 - Scen_CarSegments['shAwareEV']
Scen_CarSegments['shPNoAccessToOC'] = 1 - Scen_CarSegments['shPAccessToOC']
Scen_CarSegments['shFNoCertaintyOfAccess'] = 1 - Scen_CarSegments['shFCertaintyOfAccess']

# Define new dataframes for Utility/MarketSh, SumNewCars/MarketShTotal, NewCars
SumNewCars = pd.DataFrame(columns=['Year', 'Private_Fleet', 'Size', 'Consumer', 'Charging_Access', 'SumNewCars'])
MarketShare = pd.DataFrame(columns=['TechID', 'Year', 'Size', 'Private_Fleet', 'Consumer', 'Utility', 'MarketShare'])
NewCars = pd.DataFrame(
    columns=['LSOA', 'Year', 'TechID', 'Private_Fleet', 'Size', 'Consumer', 'Charging_Access', 'NewCars'])

#TODO: Instantiate a NEW dataframe, TotalCars, that takes the results of NewCars and adds them to *separate* age bands to allow calculation of scrappage per yr
TotalCars = pd.DataFrame(
    columns=['LSOA', 'Year', 'TechID', 'Private_Fleet', 'Size', 'Consumer', 'Charging_Access'] + [
        'TotalCars_AGE' + str(a) for a in range(22)])

#TODO: TotalCars gets passed to the return_NewCars function.

# Set possible configurations -- Private/Fleet, Size, Consumer, Charging_Access
Private_Fleet_Options = ['Private', 'Fleet']
Consumer_Segments = [['Enthusiast', 'Aspirer', 'Mass', 'Resistor'], ['UserChooser', 'FleetCar']]
Sizes = [1, 2, 3]
Charging_Access_Levels = [0, 1,
                          2]

In [5]:
import time

trial_cnt = 0
while trial_cnt < 1:

    # Start timer
    segstart = time.time()

    Segmentation_df = pd.DataFrame(
        columns=['LSOA', 'Year', 'TechID', 'Private_Fleet', 'Size', 'Consumer', 'Charging_Access', 'NewCars'])

    LSOA_cnt = 0
    #sample_LSOAs = random.sample(LSOAs, 1)
    sample_LSOAs = ['S01006998'] #this one is Aberdeen A52 6HJ (Connor's network)
    #sample_LSOAs = ['S01006861'] #this one has shFleet = 0, shPrivate != 0
    #sample_LSOAs = ['S01007460'] #this one has shFleet != 0, shPrivate != 0

    for LSOA in sample_LSOAs:
        SumNew = return_SumNew(NewCars_per_LSOA, LSOA)

        SumNewCars = return_SumNewCars(SumNewCars, LSOA, SumNew, Scen_CarSegments, Private_Fleet_Options,
                                       Consumer_Segments, Sizes, Charging_Access_Levels)

        MarketShare = return_MarketShare(MarketShare, SumNew, Technology, Cost_Data, Private_Fleet_Options,
                                         Consumer_Segments, NewCars,
                                         Scen_CarSegments)

        MarketShare_Total = return_MarketShare_Totals(LSOA, MarketShare, SumNew, Technology)

        SumNewCars = return_MarketShTot(SumNewCars, MarketShare, Technology)

        NewCars = return_NewCars(LSOA, NewCars, SumNew, SumNewCars, MarketShare, Technology, Consumer_Segments,
                                 Private_Fleet_Options, Charging_Access_Levels)

        Segmentation_df = Segmentation_df.append(NewCars[NewCars.NewCars > 0])

        LSOA_cnt += 1

        if LSOA_cnt % 10 == 0:
            print(LSOA_cnt)

    trial_cnt += 1
    print('overall time', time.time()-segstart)

Sumnew took 0.0069959163665771484
SumNewCars took 4.067742109298706
MarketShare took 98.22248125076294
marketshtot took 5.190133810043335
newcars took 115.86280393600464
overall time 230.5121841430664


In [6]:
NewCars


,LSOA,Year,TechID,Private_Fleet,Size,Consumer,Charging_Access,NewCars
0,S01006998,2012,80,Private,1,Enthusiast,0,0.204245
1,S01006998,2012,91,Private,1,Enthusiast,0,0
2,S01006998,2012,140,Private,2,Enthusiast,0,0.146084
3,S01006998,2012,161,Private,2,Enthusiast,0,0
4,S01006998,2012,191,Private,3,Enthusiast,0,0
...,...,...,...,...,...,...,...,...
13909,S01006998,2035,616,Fleet,2,FleetCar,2,0
13910,S01006998,2035,625,Fleet,1,FleetCar,2,0
13911,S01006998,2035,636,Fleet,3,FleetCar,2,0
13912,S01006998,2035,646,Fleet,3,FleetCar,2,0


In [7]:
TotalCars = pd.DataFrame(
    columns=['LSOA', 'Year', 'TechID', 'Private_Fleet', 'Size', 'Consumer', 'Charging_Access'] + [
        'TotalCars_AGE' + str(a) for a in range(22)])

In [8]:
LSOA_AgeData = AgeData[AgeData.GEO_CODE == LSOA]


In [10]:
#prelim: set up for first year (2011)
LSOA_AgeData





,GEO_CODE,AGE_0,AGE_1,AGE_2,AGE_3,AGE_4,AGE_5,AGE_6,AGE_7,AGE_8,...,AGE_14,AGE_15,AGE_16,AGE_17,AGE_18,AGE_19,AGE_20,AGE_21,AveAGE,TotalCars
33294,S01006998,0,0,53,87,82,83,75,63,53,...,25,19,14,10,6,3,2,2,8.3416,777


In [32]:
cnt = 0
base_year = 2011

for year in SumNew.Year.tolist():
    
    # retrieve the Technologies available for this year
    AvailableTech = Technology[(Technology.Availability <= year) & (Technology.Final_Year >= year)]

    TechIDs = AvailableTech.TechID.tolist()

    for P_F in Private_Fleet_Options:
        ConsumerSegs = Consumer_Segments[Private_Fleet_Options.index(P_F)]

        for Consumer in ConsumerSegs:

            for Charging_Access in Charging_Access_Levels:

                for TechID in TechIDs:
                    
                    data = NewCars[(NewCars.Year == year) & (NewCars.Private_Fleet == P_F) & (NewCars.Consumer == Consumer) & (NewCars.Charging_Access == Charging_Access) & (NewCars.TechID == TechID)]
                    
                    TotalCars.loc[cnt, ['LSOA', 'Year', 'TechID', 'Private_Fleet', 'Size', 'Consumer', 'Charging_Access']] = [LSOA, year, TechID, P_F, data.Size.item(), Consumer, Charging_Access]
                    
                    TotalCars.loc[cnt, ['TotalCars_AGE'+str(a) for a in range(22)]] = [data.NewCars.item()/NewCars[NewCars.Year == year].NewCars.sum() * element for element in LSOA_AgeData[['AGE_'+str(a) for a in range(22)]].iloc[0].tolist()]
                    
                    cnt += 1

In [33]:
TotalCars

,LSOA,Year,TechID,Private_Fleet,Size,Consumer,Charging_Access,TotalCars_AGE0,TotalCars_AGE1,TotalCars_AGE2,...,TotalCars_AGE12,TotalCars_AGE13,TotalCars_AGE14,TotalCars_AGE15,TotalCars_AGE16,TotalCars_AGE17,TotalCars_AGE18,TotalCars_AGE19,TotalCars_AGE20,TotalCars_AGE21
0,S01006998,2012,80,Private,1,Enthusiast,0,0,0,1.35312,...,0.970161,0.842509,0.638264,0.485081,0.357428,0.255306,0.153183,0.0765917,0.0510611,0.0510611
1,S01006998,2012,91,Private,1,Enthusiast,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,S01006998,2012,140,Private,2,Enthusiast,0,0,0,0.967805,...,0.693898,0.602596,0.456512,0.346949,0.255647,0.182605,0.109563,0.0547814,0.036521,0.036521
3,S01006998,2012,161,Private,2,Enthusiast,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,S01006998,2012,191,Private,3,Enthusiast,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13909,S01006998,2035,616,Fleet,2,FleetCar,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13910,S01006998,2035,625,Fleet,1,FleetCar,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13911,S01006998,2035,636,Fleet,3,FleetCar,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13912,S01006998,2035,646,Fleet,3,FleetCar,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
data = NewCars[(NewCars.Year == year) & (NewCars.Private_Fleet == P_F) & (NewCars.Consumer == Consumer) & (NewCars.Charging_Access == Charging_Access) & (NewCars.TechID == TechID)]


In [41]:
LSOA_AgeData

,GEO_CODE,AGE_0,AGE_1,AGE_2,AGE_3,AGE_4,AGE_5,AGE_6,AGE_7,AGE_8,...,AGE_14,AGE_15,AGE_16,AGE_17,AGE_18,AGE_19,AGE_20,AGE_21,AveAGE,TotalCars
33294,S01006998,0,0,53,87,82,83,75,63,53,...,25,19,14,10,6,3,2,2,8.3416,777


In [37]:
TotalCars[TotalCars.Year==2012][['TotalCars_AGE'+str(k) for k in range(22)]].sum()


TotalCars_AGE0      0.0
TotalCars_AGE1      0.0
TotalCars_AGE2     53.0
TotalCars_AGE3     87.0
TotalCars_AGE4     82.0
TotalCars_AGE5     83.0
TotalCars_AGE6     75.0
TotalCars_AGE7     63.0
TotalCars_AGE8     53.0
TotalCars_AGE9     48.0
TotalCars_AGE10    45.0
TotalCars_AGE11    36.0
TotalCars_AGE12    38.0
TotalCars_AGE13    33.0
TotalCars_AGE14    25.0
TotalCars_AGE15    19.0
TotalCars_AGE16    14.0
TotalCars_AGE17    10.0
TotalCars_AGE18     6.0
TotalCars_AGE19     3.0
TotalCars_AGE20     2.0
TotalCars_AGE21     2.0
dtype: float64